# Latin Hypercube Design 拉丁方格抽樣

In [ ]:
import random
import csv
from pyDOE import *
from scipy.stats import uniform

import pandas as pd
import numpy as np

#Vissim
import win32com.client as com
import os

In [ ]:
design = lhs(12, samples=200)
#針對CC0、CC1、CC2、CC7進行參數校估
#共有3個分段需要較估
#上下界設定可自行調整，可參考江宜穎「高速公路壅塞模擬與主線速率漸變控制模式之研究」，國立台灣大學土木工程學研究所交通組碩士論文，民國102年。
#上下界=[CC0、CC1、CC2、CC7,\ 第一段
#       CC0、CC1、CC2、CC7,\ 第二段
#       CC0、CC1、CC2、CC7]  第三段
#抽樣下界
 L=[0.5,2.3,0,0,\
   0.5,2.2,0,0,\
   0.5,1.8,0,0]
#抽樣上界
U=[3,2.6,6,1,\
   3,2.6,6,1,\
   3,2.5,6,1]
scale=np.array(U)-np.array(L)
for i in range(12):
    design[:,i] = uniform(loc=L[i], scale=scale[i]).ppf(design[:, i])
design = pd.DataFrame(design)
design = design.round(2)
design

In [ ]:
#設定檔案路徑
path='請自行設定'

In [ ]:
design.to_csv(path+r"LHD.csv",index=False)

In [ ]:
design=pd.read_csv(path+r"LHD.csv")
design

In [ ]:
#參數抽樣完成後，進行VISSIM模擬，並計算各組參數MAPE
F = pd.read_csv(path+r"流量資料")
S = pd.read_csv(path+r"速率資料")
for i in range(design.shape[0]):
    ind = design.loc[i,:].to_list()
    reset_driving_behavior(ind)
    # run simulation
    # Activate QuickMode:
    Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",1)
    Vissim.Simulation.RunContinuous()

    f, s = load_vissim_data()
    f.columns = F.columns
    s.columns = S.columns
    F_mean = round(mape(F.iloc[:,0:20],f.iloc[:,0:20]).mean(),4)*100
    S_mean = round(mape(S.iloc[:,0:20],s.iloc[:,0:20]).mean(),4)*100
    with open(r"C:\Users\hyang\Desktop\HYANG\vissim com\LHD\exp4\LHD_history.csv", 'a', newline='') as csvfile:
        # 定義欄位
        colname = ['fCC0', 'fCC1', 'fCC2', 'fCC7', 'mCC0', 'mCC1', 'mCC2', 'mCC7', 'eCC0', 'eCC1', 'eCC2', 'eCC7', 'fmape', 'smape']
        # 將 dictionary 寫入 CSV 檔
        writer = csv.DictWriter(csvfile, fieldnames=colname)
        # 寫入第一列的欄位名稱
        #writer.writeheader()
        temp = ind
        temp.append(F_mean)
        temp.append(S_mean)
        writer.writerow({k:i for k, i in zip(colname, temp)})

In [ ]:
#存檔並選出績效最好的前10組參數
LHD_result = pd.read_csv(path+r"LHD_history.csv")
LHD_result = LHD_result.sort_values(by='smape')
LHD_result.head(10)